# Elements of Programming Interviews
## Heaps
### Track 7: 11.1, 11.3, 11.4, 11.5, 11.7

### 11.1 - Merge Sorted Files

Write a program that takes as input a set of sorted sequences and computes the union of these sequences as a sorted sequence. 

For example, if the input is $[{3,5,7}],[{0,6}],[{0,6,28}]$ then the output is $[{0,0,3,5,6,6,7,28}]$

>Brute force approach is to have a tracker index for each sequence, each starting at index 0. Then find the minimum element out of all the tracker indices, push that element into the sorted sequence and increment the respective tracker index.


In [1]:
import heapq

In [2]:
def add_to_heap(heap, list_num, it):
    try:
        heapq.heappush(heap, (next(it), list_num))
    except StopIteration:
        pass
def merge_k_sorted_lists(*sequences):
    iters = list(map(iter, sequences))
    heap = []
    res = []
    #init heap to first vals of each seq
    #each heap elem holds (min_val, heap#)
    for seq_num, it in enumerate(iters):
        add_to_heap(heap, seq_num, it)
    while heap:
        min_val, seq_num = heapq.heappop(heap)
        add_to_heap(heap, seq_num, iters[seq_num])
        res.append(min_val)
    return res

In [3]:
sequences = [[0,3,5], [1, 2, 4]]
merge_k_sorted_lists([0, 2, 4, 6], [1, 7], [3, 5, 8])

[0, 1, 2, 3, 4, 5, 6, 7, 8]

### 11.3 - Sort An Almost Sorted Array
>Write a program which takes as input a very long sequence of numbers. Each number is at most $k$ away from its correctly sorted position. For exmaple, no number in the sequence
>>$ { 3, -1, 2, 6, 4, 5, 8 } $

>is more than 2 away from its final sorted position. Design an algorithm that output the numbers in the correct order.

The key insight here is to find the $k$ value. Once you have that, you can have a rolling heap of size $k$ iterating over the array, placing the minimum values accordingly. 

In [31]:
def find_k(arr):
    #for each unsorted elem, find the max # of moves 
    #it is away from correct spot
    max_k = 0
    for i in range(len(arr)-1):
        if arr[i] > arr[i+1]:
            k = 0
            unsorted_elem = arr[i]
            while (i+1) < len(arr) and unsorted_elem > arr[i + 1]:
                k += 1
                i += 1
            max_k = k if k > max_k else max_k
    return max_k
from IPython.core.debugger import Tracer
def sort_almost_sorted(arr):
    heap =[]
    k = find_k(arr)
    #init heap to first k vals
    for elem in arr[:k]:
        heapq.heappush(heap, elem)
    for i in range(len(arr) - k):
        arr[i] = heapq.heappop(heap)
        heapq.heappush(heap, arr[i + k])
    for i in range(len(arr)-k, len(arr)):
        arr[i] = heapq.heappop(heap)
    return arr

In [33]:
almost_sorted = [3,-1,2,6,4,5,8]
alm = [2,-1,6,4,5,10,7,8,9]
find_k(almost_sorted), find_k(alm)
sort_almost_sorted(almost_sorted), sort_almost_sorted([5,10,7,8,9])

([-1, 2, 3, 4, 5, 6, 8], [5, 7, 8, 9, 10])